In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mne_connectivity import spectral_connectivity_time

In [14]:
n_times = 128


n_cycles=7
sfreq = 32
dur = float(n_times) / sfreq
cycle_freq = n_cycles / dur
print('duration: %s' % dur)
print('cycle freq: %s' % cycle_freq)

# minimum frequency possible:
m = (n_cycles * sfreq) / n_times
print('minimum frequency possible: %s' % m)

freqs = np.arange(14, 50, 1)
np.any(freqs < cycle_freq)

duration: 4.0
cycle freq: 1.75
minimum frequency possible: 1.75


np.False_

In [13]:


def sinusoidal(a, f, sr, t, theta=0, DC=0):
    delta_i = 1 / sr
    f2pi = f * 2 * np.pi
    nu = np.array([DC + (a * np.sin(f2pi * i * delta_i + theta)) for i in range(t)])
    return nu
t=32
sr=32
f=2
noise=1000

v=sinusoidal(10, f, sr, t*4, 0)
w=sinusoidal(10, f, sr, t*4, np.pi/4)
y=sinusoidal(10, f, sr, t*4, np.pi/2)
z=sinusoidal(10, f, sr, t*4, np.pi)
# need an array shaped epoch, channel, time
data = np.array([[v, w, y, z]])
data.shape


(1, 4, 128)

In [20]:
conn = spectral_connectivity_time(
    data,
    freqs=np.arange(2,16),
    method='coh',
    sfreq=sr,
    # faverage=True,
    verbose=True,
    # n_jobs=20,
    # indices=indices,
    mode="multitaper",
)

Fmin was not specified. Using fmin=min(freqs)
Fmax was not specified. Using fmax=max(freqs).
only using indices for lower-triangular matrix
Connectivity computation...
   Processing epoch 1 / 1 ...
[Connectivity computation done]


In [21]:
out = conn.get_data(output='dense')
out.shape

(1, 4, 4, 14)

In [23]:
out[0,:,:,2]

array([[0.        , 0.        , 0.        , 0.        ],
       [0.89411542, 0.        , 0.        , 0.        ],
       [0.85173113, 0.94322649, 0.        , 0.        ],
       [1.        , 0.89411542, 0.85173113, 0.        ]])

We have 
- a set of matrices where each represents the connectivity of a graph. 
    - can be directed or undirected
    - can be in time, as as a windowed function or some how pooled over time
- a corresponding set of x,y,z coordinates for each node in the graph representing the position of the node in 3D MNI space.
- a set of labels for each node in the graph representing if the node is EZ or not.

In [1]:
import mne
import numpy as np
from mne import make_fixed_length_epochs
from mne_connectivity import spectral_connectivity_time

from matplotlib import pyplot as plt

In [2]:
path = "/media/dan/Data/data/iEEG/raw_ieeg/baseline_patients/baseline_edfs/064_Baseline.EDF"
raw = mne.io.read_raw_edf(path,verbose=False)
eeg_channels = [ch for ch in raw.ch_names if ch.startswith("EEG ")]
raw = mne.io.read_raw_edf(path, preload=True, verbose=False,include=eeg_channels)
data = raw.get_data()

/tmp/ipykernel_28592/2729598758.py:2: RuntimeWarning: Omitted 6 annotation(s) that were outside data range.
  raw = mne.io.read_raw_edf(path,verbose=False)
/tmp/ipykernel_28592/2729598758.py:4: RuntimeWarning: Omitted 6 annotation(s) that were outside data range.
  raw = mne.io.read_raw_edf(path, preload=True, verbose=False,include=eeg_channels)


In [3]:
epochs = make_fixed_length_epochs(raw=raw, duration=.5)
times = epochs.times
ch_names = epochs.ch_names

# conn = vector_auto_regression(data=epochs.get_data(), times=times, names=ch_names, n_jobs=25)

# # this returns a connectivity structure over time
# print(conn)

Not setting metadata
602 matching events found
No baseline correction applied
0 projection items activated


In [4]:
epochs.get_data().shape

Using data from preloaded Raw for 602 events and 1024 original time points ...
0 bad epochs dropped


(602, 194, 1024)

In [5]:
n_times = 1024


n_cycles=7
sfreq = raw.info['sfreq']
dur = float(n_times) / sfreq
cycle_freq = n_cycles / dur
print('duration: %s' % dur)
print('cycle freq: %s' % cycle_freq)

# minimum frequency possible:
m = (n_cycles * sfreq) / n_times
print('minimum frequency possible: %s' % m)

freqs = np.arange(14, 50, 1)
np.any(freqs < cycle_freq)

duration: 0.5
cycle freq: 14.0
minimum frequency possible: 14.0


np.False_

In [ ]:
conn = []

# indexs is a tuple of two lists, each containing the indices of the channels to be connected by index
# compute all possible connection combinations
N = len(raw.ch_names)
# N = 4
indices = np.array([(i, j) for i in range(N) for j in range(N)])
indices = (indices[:, 0], indices[:, 1]) # make list of tuples for MNE connectivity functions 


for method in ['coh', 'plv', 'ciplv', 'pli', 'wpli']:
    conn.append(
        spectral_connectivity_time(
            epochs.get_data()[:5, :, :],
            freqs=np.arange(14,300),
            method=method,
            sfreq=raw.info["sfreq"],
            faverage=True,
            verbose=True,
            n_jobs=20,
            indices=indices,
            mode="multitaper",
        )
    )

conn = np.array(conn)

Using data from preloaded Raw for 602 events and 1024 original time points ...
Fmin was not specified. Using fmin=min(freqs)
Fmax was not specified. Using fmax=max(freqs).
Connectivity computation...
   Processing epoch 1 / 5 ...


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  52 tasks      | elapsed:    3.9s
[Parallel(n_jobs=20)]: Done 142 tasks      | elapsed:    7.5s
[Parallel(n_jobs=20)]: Done 194 out of 194 | elapsed:    9.3s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  52 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Done 204 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done 456 tasks      | elapsed:    5.2s
[Parallel(n_jobs=20)]: Done 780 tasks      | elapsed:    7.2s
[Parallel(n_jobs=20)]: Done 1176 tasks      | elapsed:    9.7s
[Parallel(n_jobs=20)]: Done 1644 tasks      | elapsed:   12.5s
[Parallel(n_jobs=20)]: Done 2184 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 2796 tasks      | elapsed:   19.2s
[Parallel(n_jobs=20)]: Done 3480 tasks      | elapsed:   23.3s
[Parallel(n_jobs=20)]: Done 4236 tasks      | elapsed:   27.7s
[Parallel(n_jobs=20)]: Do

   Processing epoch 2 / 5 ...


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  84 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 194 out of 194 | elapsed:    5.0s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  52 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 204 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done 456 tasks      | elapsed:    5.3s
[Parallel(n_jobs=20)]: Done 780 tasks      | elapsed:    7.3s
[Parallel(n_jobs=20)]: Done 1176 tasks      | elapsed:    9.8s
[Parallel(n_jobs=20)]: Done 1644 tasks      | elapsed:   12.5s
[Parallel(n_jobs=20)]: Done 2184 tasks      | elapsed:   15.7s
[Parallel(n_jobs=20)]: Done 2796 tasks      | elapsed:   19.5s
[Parallel(n_jobs=20)]: Done 3480 tasks      | elapsed:   23.7s
[Parallel(n_jobs=20)]: Done 4236 tasks      | elapsed:   28.3s
[Parallel(n_jobs=20)]: Done 5064 tasks      | elapsed:   33.2s
[Parallel(n_jobs=20)]: D

   Processing epoch 3 / 5 ...


...

[Parallel(n_jobs=20)]: Done 37636 out of 37636 | elapsed:  4.0min finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  84 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done 194 out of 194 | elapsed:    5.1s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  52 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 204 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 456 tasks      | elapsed:    5.0s
[Parallel(n_jobs=20)]: Done 780 tasks      | elapsed:    7.0s
[Parallel(n_jobs=20)]: Done 1176 tasks      | elapsed:    9.6s
[Parallel(n_jobs=20)]: Done 1644 tasks      | elapsed:   12.6s
[Parallel(n_jobs=20)]: Done 2184 tasks      | elapsed:   16.2s
[Parallel(n_jobs=20)]: Done 2796 tasks      | elapsed:   20.0s
[Parallel(n_jobs=20)]: Done 3480 tasks      | elapsed:   24.3s
[Parallel(n_jobs=20)]: Done 4236 tasks      | elapsed:   29.4s
[Parallel(n_

   Processing epoch 4 / 5 ...


[Parallel(n_jobs=20)]: Done 37636 out of 37636 | elapsed:  4.0min finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  84 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 194 out of 194 | elapsed:    5.0s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  52 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 204 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 456 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 780 tasks      | elapsed:    6.8s
[Parallel(n_jobs=20)]: Done 1176 tasks      | elapsed:    9.4s
[Parallel(n_jobs=20)]: Done 1644 tasks      | elapsed:   12.3s
[Parallel(n_jobs=20)]: Done 2184 tasks      | elapsed:   15.8s
[Parallel(n_jobs=20)]: Done 2796 tasks      | elapsed:   19.5s
[Parallel(n_jobs=20)]: Done 3480 tasks      | elapsed:   23.7s
[Parallel(n_jobs=20)]: Done 4236 tasks      | elapsed:   28.4s
[Parallel(n_

   Processing epoch 5 / 5 ...


[Parallel(n_jobs=20)]: Done 37636 out of 37636 | elapsed:  4.0min finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done 108 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Done 194 out of 194 | elapsed:    5.3s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  52 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 204 tasks      | elapsed:    3.2s
[Parallel(n_jobs=20)]: Done 456 tasks      | elapsed:    4.7s
[Parallel(n_jobs=20)]: Done 780 tasks      | elapsed:    6.7s
[Parallel(n_jobs=20)]: Done 1176 tasks      | elapsed:    9.2s
[Parallel(n_jobs=20)]: Done 1644 tasks      | elapsed:   12.2s
[Parallel(n_jobs=20)]: Done 2184 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 2796 tasks      | elapsed:   19.3s
[Parallel(n_jobs=20)]: Done 3480 tasks      | elapsed:   23.3s
[Parallel(n_jobs=20)]: Done 4236 tasks      | elapsed:   28.2s
[Parallel(n_

In [84]:
for i, method in enumerate(['coh', 'plv', 'ciplv', 'pli', 'wpli']):
    print(f"Method: {method}")
    print(conn[i].get_data(output='dense').shape)

Method: coh
(5, 194, 194, 1)
Method: plv
(5, 194, 194, 1)
Method: ciplv
(5, 194, 194, 1)
Method: pli
(5, 194, 194, 1)
Method: wpli
(5, 194, 194, 1)


In [ ]:
'cacoh', 'mic', 'mim', 'gc', 'gc_tr'